In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Paths
data_dir = 'data'
csv_path = 'indian_dishes_nutritional_values.csv'

# Load nutrition data
nutri_df = pd.read_csv(csv_path)
nutri_dict = nutri_df.set_index("Dish Name").to_dict(orient="index")

# Data generator
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Pretrained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model for initial training
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Fine-tune top layers
for layer in base_model.layers[-40:]:  # Unfreeze last 40 layers
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10)

model.save('fine_tuned_mobilenetv2_food_model.h5')


Found 3200 images belonging to 80 classes.
Found 800 images belonging to 80 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


c:\Users\Bennett Varghese\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 435ms/step - accuracy: 0.0177 - loss: 4.4994 - val_accuracy: 0.0425 - val_loss: 4.2832
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 470ms/step - accuracy: 0.0658 - loss: 4.1508 - val_accuracy: 0.0787 - val_loss: 4.0388
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 47s 466ms/step - accuracy: 0.1407 - loss: 3.8355 - val_accuracy: 0.1488 - val_loss: 3.7249
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 473ms/step - accuracy: 0.2288 - loss: 3.4568 - val_accuracy: 0.2212 - val_loss: 3.4033
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 476ms/step - accuracy: 0.3278 - loss: 3.0391 - val_accuracy: 0.2600 - val_loss: 3.1217
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 49s 481ms/step - accuracy: 0.3925 - loss: 2.6962 - val_accuracy: 0.3137 - val_loss: 2.8796
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 499ms/step - accuracy: 0.4509 - loss: 2.4655 - val_accuracy: 0.3562 - val_loss: 2.6962
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 501ms/step - accuracy: 0.5198 - loss: 2.1901 - val

In [12]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os


# --- Paths ---
model_path = 'fine_tuned_mobilenetv2_food_model.h5'
test_image_path = 'data/sutar_feni/2e2f921aa6.jpg'  # Replace with your test image
csv_path = 'indian_dishes_nutritional_values.csv'
img_size = (224, 224)

# --- Load model and data ---
model = load_model(model_path)
nutri_df = pd.read_csv(csv_path)
nutri_dict = nutri_df.set_index("Dish Name").to_dict(orient="index")

# --- Load class labels ---
class_labels = sorted(os.listdir('data'))

# --- Predict image class ---
img = load_img(test_image_path, target_size=img_size)
x = img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)
pred = model.predict(x)
label = class_labels[np.argmax(pred)]

# --- Get nutrition info ---
nutrition = nutri_dict.get(label, "Nutrition data not found")

# --- Output result ---
print(f"Predicted Dish: {label}")
print("Nutritional Information:", nutrition)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 829ms/step
Predicted Dish: sutar_feni
Nutritional Information: {'Calories(kcal)': 320, 'Carbohydrates(g)': 34, 'Protein(g)': 3, 'Fats(g)': 18, 'Sugar(g)': 25}
